In [0]:
!pip install torch
!pip install torchvision
!pip install Pillow==4.0.0
!pip install -U -q PyDrive

In [0]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from PIL import Image
import torch.nn as nn
from zipfile import ZipFile
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
%matplotlib inline

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
listed = drive.ListFile().GetList()
for file in listed:
    print('title {}, id {}'.format(file['title'], file['id']))

In [0]:
downloaded = drive.CreateFile({'id': '11j1LyGLuq-HIHEgPNlzcxg0pO57rBA2y'})
downloaded.GetContentFile('train.zip')
with ZipFile("train.zip", 'r') as z:
    z.extractall()
os.remove("train.zip")

In [0]:
class TGSSaltTrainDataset(Dataset):
    def __init__(self, image_dir,mask_dir,depth_csv,train_csv):
        self.image_dir=image_dir
        self.mask_dir=mask_dir
        
#         depth=pd.read_csv(depth_csv)
#         depth["z"]=(depth["z"]-depth["z"].min())/(depth["z"].max()-depth["z"].min())
        self.filter = np.array([(0,-1,-1,-1),(1,0,0,0),(1,0,1,0),(0,1,0,1)])/8
        self.input = pd.read_csv(train_csv)
        self.input['z'] = (self.input['z']-self.input['z'].min())/(self.input['z'].max()-self.input['z'].min())
        self.input.drop(['rle_mask'],axis=1,inplace=True)
#         self.input = self.input.merge(depth,how="left",on="id")
        
        
    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir,self.input.iloc[idx,0]+".png")
        img = cv2.imread(img_name)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        img = torch.tensor(img).view(101,101).float()
      
        mask_name = os.path.join(self.mask_dir,self.input.iloc[idx,0]+".png")
        mask = cv2.imread(mask_name)
        mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
        mask = torch.tensor(mask).float()/255
        depth = self.input.iloc[idx,1].reshape(1)
        return img,mask,depth

In [0]:
train_dataset = TGSSaltTrainDataset("train/images","train/masks","depths.csv","train.csv")

In [0]:
class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x


class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x


class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True,up_size=None):
        super(up, self).__init__()

        #  would be a nice idea if the upsampling could be learned too,
        #  but my machine do not have enough memory to handle all those weights
        if bilinear:
            if up_size:
                self.up = nn.Upsample(size= up_size, mode='bilinear', align_corners=True)
            else:
                self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, in_ch//2, 2, stride=2)

        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffX = x1.size()[2] - x2.size()[2]
        diffY = x1.size()[3] - x2.size()[3]
        x2 = F.pad(x2, (diffX // 2 , int(diffX / 2),
                        diffY // 2, int(diffY / 2)))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x



class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x
class conv3d(nn.Module):
    '''
    Takes a image with given number of channels and performs 3d convolution and then returns the image as 2d
    '''
    def __init__(self):
        super(conv3d, self).__init__()
        self.conv1 = nn.Conv3d(1, 4, 2,stride=2,padding=1)
        self.conv2 = nn.Conv3d(4, 16 ,4,stride=2,padding=1)

    def forward(self, x):
        x.unsqueeze_(1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(-1,512,6,6)
        return x

class DN(nn.Module):
    '''
    @IN
    img: A tensor representing the grayscaled image; shape:[batch_size,1,101,101]
    d: The z index of the image
    
    @OUT
    shape:[batch_size,1,101,101] tensor representing the features found by kernel(3*3) created from depth value
    '''
    def __init__(self):
        super(DN,self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(4,9,bias=False),
                                   nn.ReLU())
        self.layer2 = nn.Sequential(nn.Linear(9,9,bias=True),
                                   nn.Sigmoid())
        self.conv1 = nn.Conv2d(1,1,kernel_size=3,stride=1,padding=1,bias=False)
        self.non_linear1 = nn.ReLU()
    def forward(self,x,d):
        d= d.view(1)
        ker = torch.tensor([d,d**2,d**3,1]).float().cuda()
#         print(ker)
        ker = self.layer1(ker)
        ker = self.layer2(ker).view(1,1,3,3)
        self.conv1.weight = nn.Parameter(ker)
        x = self.non_linear1(self.conv1(x))
        return x
class UNet(nn.Module):
    def __init__(self,debug=False):
        super(UNet, self).__init__()
        self.debug = debug
        self.inc = inconv(1, 64)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.down3 = down(256, 512)
        self.down4 = down(512, 512)
        self.up1 = up(1024, 256,up_size = (12,12))
        self.up2 = up(512, 128,up_size = (25,25))
        self.up3 = up(256, 64,up_size = (50,50))
        self.up4 = up(128, 64,up_size = (101,101))
        self.outc = outconv(64, 1)

    def forward(self, x):
        x.unsqueeze_(1)
        if self.debug: print(x.shape)
        x1 = self.inc(x)
        if self.debug: print(x1.shape)
        x2 = self.down1(x1)
        if self.debug: print(x2.shape)
        x3 = self.down2(x2)
        if self.debug: print(x3.shape)
        x4 = self.down3(x3)
        if self.debug: print(x4.shape)
        x5 = self.down4(x4)
        if self.debug: print(x5.shape)
        if self.debug: print('before up')
        x = self.up1(x5, x4)
        if self.debug: print(x.shape)
        x = self.up2(x, x3)
        if self.debug: print(x.shape)
        x = self.up3(x, x2)
        if self.debug: print(x.shape)
        x = self.up4(x, x1)
        if self.debug: print(x.shape)
        x = self.outc(x)
        if self.debug: print(x.shape)
        return x.squeeze(1)

In [0]:
class UDNet(nn.Module):
    def __init__(self):
        super(UDNet,self).__init__()
#         self.dn = DN()
#         self.un1 = UNet(debug=True)
#         self.un2 = UNet(debug=True)
        self.un1 = UNet(debug=False)
#         self.un2 = UNet(debug=False)
    def forward(self,x,d):
        x = self.un1(x)
#         x.unsqueeze_(1)
#         x = self.dn(x,d)
#         x.squeeze_(1)
#         x = self.un2(x)
        return x

In [99]:
model = UDNet().cuda()
x = torch.randn(10,101,101).cuda()
print(model(x,torch.tensor(train_dataset[1][2][0])).shape)

torch.Size([10, 101, 101])


In [0]:
batch_size = 100
validation_split = 0.2
shuffle_dataset = True
random_seed= 42
# Creating data indices for training and validation splits:
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,sampler=validation_sampler)

In [0]:
class BinaryCrossEntropyLoss2d(nn.Module):
    def __init__(self, weight=None, size_average=True):
        """
        Binary cross entropy loss 2D
        Args:
            weight:
            size_average:
        """
        super(BinaryCrossEntropyLoss2d, self).__init__()
        self.bce_loss = nn.BCELoss(weight, size_average)
        if torch.cuda.is_available():
            self.bce_loss = self.bce_loss.cuda()

    def forward(self, pred, target):
        pred = F.sigmoid(pred)
        pred = pred.view(-1)  # Flatten
        target = target.view(-1)  # Flatten
        return self.bce_loss(pred, target)

In [0]:
class SoftDiceLoss(nn.Module):
    def __init__(self):
        super(SoftDiceLoss, self).__init__()
    def forward(self, pred, target):
        smooth = 1
        num = target.size(0)
        pred = F.sigmoid(pred)
        pred = pred.view(num, -1)
        target = target.view(num, -1)
        intersection = (pred * target)
        score = 2. * (intersection.sum(1) + smooth) / (pred.sum(1) + target.sum(1) + smooth)
        score = 1 - score.sum() / num
        return score

In [0]:
def dice_coeff(pred, target):
    smooth = 1.
    num = target.size(0)
    pred = pred.view(num, -1)  # Flatten
    target = target.view(num, -1)  # Flatten
    intersection = (pred * target)
    score = (2. * intersection.sum(1) + smooth).float() / (pred.sum(1) + target.sum(1) + smooth).float()
    return score.sum()/num

In [0]:
def bce_dice_loss(y_true, y_pred):
    return 0.5*BinaryCrossEntropyLoss2d()(y_true, y_pred)-dice_coeff(y_true, y_pred)

In [0]:
model=UDNet()
criterion = SoftDiceLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
if torch.cuda.is_available():
    model = model.cuda()

In [0]:
def validate(threshold):
    total_loss = 0
    accuracy = 0
    model.eval()
    for batch_idx, (data,target,d) in enumerate(validation_loader):
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        # forward
        output = model(data,d)
        predict = (F.sigmoid(output) > threshold).float()
        # backward + optimize
        loss = criterion(predict, target)
        # print statistics
        accuracy += dice_coeff(predict, target).item()
        total_loss+=loss.item()
    print('Validation Loss: {:.5f} Validation Accuracy: {:.5f}'.format(total_loss*batch_size/len(val_indices),accuracy*batch_size/len(val_indices)))

In [0]:
def train(threshold):
    epoch=1
    while True:
        total_loss = 0
        total_accuracy = 0
        model.train()
        exp_lr_scheduler.step()
        print(exp_lr_scheduler.get_lr())
        for batch_idx, (data,target,d) in enumerate(train_loader):
            if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()
            # forward
            output = model(data,d)
            predict = (F.sigmoid(output) > threshold).float()
            # backward + optimize
            loss = criterion(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # print statistics
            accuracy = dice_coeff(predict, target)
            total_accuracy+=accuracy
            total_loss+=loss
            print('Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.5f}\tAccuracy: {:.5f}'.format(epoch, (batch_idx + 1) * len(data), len(train_indices),100*(batch_idx + 1)* len(data) / len(train_indices), total_loss.item(),accuracy))
#             if batch_idx%5 == 0:
#                 print(str(batch_idx/len(train_indices)*100)+'% completed')
    
#              print('Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.5f}\tAccuracy: {:.5f}'.format(epoch, (batch_idx + 1) * len(data), len(train_indices),100*(batch_idx + 1)* len(data) / len(train_indices), total_loss.item(),accuracy))
        print('Train Loss: {:.5f} Train Accuracy: {:.5f}'.format(total_loss.item()*batch_size/len(train_indices),total_accuracy.item()*batch_size/len(train_indices)))
        validate(threshold)
        epoch+=1

In [0]:
train(0.5)

[0.01]
Epoch: 1 [100/12800 (1%)]	Loss: 0.80632	Accuracy: 0.13122
Epoch: 1 [200/12800 (2%)]	Loss: 1.58609	Accuracy: 0.22974
Epoch: 1 [300/12800 (2%)]	Loss: 2.34998	Accuracy: 0.29097
Epoch: 1 [400/12800 (3%)]	Loss: 3.07022	Accuracy: 0.33257
Epoch: 1 [500/12800 (4%)]	Loss: 3.83589	Accuracy: 0.26247
Epoch: 1 [600/12800 (5%)]	Loss: 4.54037	Accuracy: 0.31652
Epoch: 1 [700/12800 (5%)]	Loss: 5.23666	Accuracy: 0.32281
Epoch: 1 [800/12800 (6%)]	Loss: 5.95245	Accuracy: 0.30366
Epoch: 1 [900/12800 (7%)]	Loss: 6.69539	Accuracy: 0.27105
Epoch: 1 [1000/12800 (8%)]	Loss: 7.42679	Accuracy: 0.26889
Epoch: 1 [1100/12800 (9%)]	Loss: 8.12384	Accuracy: 0.31241
Epoch: 1 [1200/12800 (9%)]	Loss: 8.82235	Accuracy: 0.29893
Epoch: 1 [1300/12800 (10%)]	Loss: 9.49604	Accuracy: 0.33111
Epoch: 1 [1400/12800 (11%)]	Loss: 10.24869	Accuracy: 0.24748
Epoch: 1 [1500/12800 (12%)]	Loss: 10.98565	Accuracy: 0.26493
Epoch: 1 [1600/12800 (12%)]	Loss: 11.68647	Accuracy: 0.29414
Epoch: 1 [1700/12800 (13%)]	Loss: 12.33737	Accuracy

In [0]:
model.cpu()
fig, axeslist = plt.subplots(ncols=5,nrows=1)

x,y =train_dataset[100]
axeslist.ravel()[4].imshow(x.detach().numpy().reshape(101,101,3))
axeslist.ravel()[0].imshow(x.detach().numpy()[0],cmap='gray')
print(x.shape)
z = model(x.view(1,3,101,101))
axeslist.ravel()[1].imshow(z.squeeze(0).detach().numpy(),cmap='gray')
print(z.shape)
def getImg(x):
    x = x.view(1,3,101,101)
    x = model(x).detach().squeeze(0).numpy()[0]
    x = (x-x.mean()/(x.max()-x.min())) +1
    a = np.expand_dims(x, axis = 2)
    a = np.concatenate((a, a, a), axis = 2)
    return a
print(y.shape)
predict = (F.sigmoid(z) > 0.5).detach().numpy().squeeze(0)
axeslist.ravel()[3].imshow(predict,cmap='gray')


# axeslist.ravel()[1].imshow(getImg(y),cmap='gray')

axeslist.ravel()[2].imshow(y.detach().numpy(),cmap='gray')
plt.show()

In [0]:
|x = torch.randn(1,3,101,101)
plt.imshow(x.numpy().squeeze(0)[0])

In [0]:
torch.save(model,'TGSUNetModel61.pt')
uploaded = drive.CreateFile({'title': 'TGSUNetModel61.pt'})
uploaded.SetContentFile('TGSUNetModel61.pt')
uploaded.Upload()

In [0]:
downloaded = drive.CreateFile({'id': '1Gs1AlcVL9WNlgHQPSpNvbFtSbqNFZIdu'})
downloaded.GetContentFile('test.zip')
with ZipFile("test.zip", 'r') as z:
  z.extractall()
os.remove("test.zip")

In [0]:
class TGSSaltTestDataset(Dataset):
    def __init__(self, image_dir,test_csv):
        self.image_dir=image_dir
        self.filter = np.array([(0,-1,-1,-1),(1,0,0,0),(1,0,1,0),(0,1,0,1)])/8
        self.input = pd.read_csv(test_csv)
        
    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir,self.input.iloc[idx,0]+".png")
        img = cv2.imread(img_name)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img1 = cv2.filter2D(img,-1,self.filter)
        
        sigma=0.33
        
        
        v = np.median(img1)

        #---- apply automatic Canny edge detection using the computed median----
        lower = int(max(0, (1.0 - sigma) * v))
        upper = int(min(255, (1.0 + sigma) * v))
        img1 = cv2.Canny(img1, lower, upper)
        
        img2 = (cv2.Laplacian(img, cv2.CV_32F) + 127.0)
#         img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
        
        img = torch.tensor(img).view(1,101,101).float()
        img1 = torch.tensor(img1).view(1,101,101).float()
        img2 = torch.tensor(img2).view(1,101,101)
        
        img = torch.cat((img,img1,img2),dim=0)/255
        
        return img

In [0]:
batch_size = 50
test_dataset = TGSSaltTestDataset("test/images","test.csv")
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [0]:
def predict(test_loader):
    test_pred=torch.IntTensor().cuda()
    model.eval()
    for batch_idx, data in enumerate(test_loader):
        if torch.cuda.is_available():
            data = data.cuda()
#             target = target.cuda()
        # forward
        output = model(data)
        predictx = (F.sigmoid(output) > 0.5).int()
        test_pred=torch.cat((test_pred,predictx.view(batch_size,101,101)),dim=0)
    return test_pred.cpu().numpy()

In [0]:
predict=prediciton(test_loader)
predict=predict.cpu().numpy()

In [0]:
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [0]:
test_dataset.input.drop("z",axis=1,inplace=True)
predicted = predict(test_loader)
test_dataset.input['rle_mask']=np.nan
for i in range(len(test_dataset)):
    test_dataset.input["rle_mask"][i]=rle_encode(predicted[i])

In [0]:
test_dataset.input.to_csv("submission11.csv",index=False)
uploaded = drive.CreateFile({'title': 'submission11.csv'})
uploaded.SetContentFile('submission11.csv')
uploaded.Upload()

from https://github.com/milesial/Pytorch-UNet/blob/master/unet/unet_model.py

In [4]:
torch.__version__

'0.4.1'

In [70]:
torch.tensor(train_dataset[1][2][0])

tensor(0.7608)